In [1]:
import autograd
from autograd import numpy as np

In [2]:
from collections import OrderedDict
import paragami

In [3]:
import time

a = np.full(100000, float('nan'))
b = np.random.random(a.shape)

def set_inplace(b, a):
    a[:] = b + 2
    return a

def set_notinplace(b):
    return b + 2

set_inplace(b, a)

t1 = time.time()
for iter in range(1000):
    set_inplace(b, a)
print('Inplace: ', time.time() - t1)

t1 = time.time()
for iter in range(1000):
    a = set_notinplace(b)
print('Not in place: ', time.time() - t1)

# WTF?  Try %memit and id

Inplace:  0.1567983627319336
Not in place:  0.0527186393737793


In [4]:


def check_dict_equal(dict1, dict2):
    assert dict1.keys() == dict2.keys()
    for key  in dict1:
        assert_array_almost_equal(dict1[key], dict2[key])

dict_pattern = paragami.PatternDict()
dict_pattern['a'] = paragami.NumericArrayPattern((2, 3, 4), lb=-1, ub=2)
dict_pattern['b'] = paragami.NumericArrayPattern((5, ), lb=-1, ub=10)
dict_pattern['c'] = paragami.NumericArrayPattern((5, 2), lb=-1, ub=10)

dict_val = dict_pattern.random()
print(dict_val)
dict_val2 = dict(dict_val)
print(dict_val2)
flat_val1 = dict_pattern.flatten(dict_val, free=True)
flat_val2 = dict_pattern.flatten(dict_val2, free=True)

np.linalg.norm(flat_val1 - flat_val2)

OrderedDict([('a', array([[[0.66437677, 0.89573098, 0.53377016, 0.86179103],
        [0.81194044, 0.87549005, 0.80827975, 0.97037133],
        [0.84310819, 1.02873991, 0.73023418, 0.58747051]],

       [[1.0063148 , 1.17924352, 0.94890452, 0.53149581],
        [0.9008666 , 0.99752516, 1.10443646, 1.04338153],
        [0.9861887 , 0.86280289, 0.8387656 , 0.83128506]]])), ('b', array([4.51412276, 6.68489851, 5.0041398 , 7.02296879, 6.21435241])), ('c', array([[5.92897591, 6.304292  ],
       [5.55899665, 4.57259256],
       [4.50770571, 5.34007301],
       [6.39642929, 6.37029544],
       [7.02907154, 4.61861735]]))])
{'b': array([4.51412276, 6.68489851, 5.0041398 , 7.02296879, 6.21435241]), 'a': array([[[0.66437677, 0.89573098, 0.53377016, 0.86179103],
        [0.81194044, 0.87549005, 0.80827975, 0.97037133],
        [0.84310819, 1.02873991, 0.73023418, 0.58747051]],

       [[1.0063148 , 1.17924352, 0.94890452, 0.53149581],
        [0.9008666 , 0.99752516, 1.10443646, 1.04338153],
    

0.0

In [5]:
param_dict = dict_pattern.random()

def test_function(param_dict):
    a = param_dict['a']
    b = param_dict['b']
    return np.mean(a ** 2) + np.mean(b ** 2)

test_function(param_dict)

39.604015332871924

In [6]:
class PatternedFunction:
    def __init__(self, original_fun, pattern, free, argnum=0):
        self._fun = original_fun
        self._argnum = argnum
        self.free = free
        self._pattern = pattern
        
    def __str__(self):
        return('Function: {}\nargnum: {}\nfree: {}\npattern: {}'.format(
            self._fun, self._argnum, self.free, self._pattern))
    
    def __call__(self, *args, **kwargs):
        flat_val = args[self._argnum]
        folded_val = self._pattern.fold(flat_val, free=self.free)
        new_args = args[0:self._argnum] + (folded_val, ) + args[self._argnum + 1:-1]
        return self._fun(*new_args, **kwargs)

    
patterned_test_function = PatternedFunction(test_function, dict_pattern, True)
print(patterned_test_function)

flat_param_dict = dict_pattern.flatten(param_dict, free=True)
assert(np.abs(patterned_test_function(flat_param_dict) - test_function(param_dict)) < 1e-12)

patterned_test_function_grad = autograd.grad(patterned_test_function)
patterned_test_function_grad(flat_param_dict)

patterned_test_function_hessian = autograd.hessian(patterned_test_function)
patterned_test_function_hessian(flat_param_dict)

Function: <function test_function at 0x7f362a34c9d8>
argnum: 0
free: True
pattern: OrderedDict:
	[a] = Array (2, 3, 4) (lb=-1, ub=2)
	[b] = Array (5,) (lb=-1, ub=10)
	[c] = Array (5, 2) (lb=-1, ub=10)


array([[0.01973457, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04455786, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.03807713, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [7]:
class FlattenedFunction:
    def __init__(self, original_fun, patterns, free, argnums=None):
        self._fun = original_fun
        self._patterns = np.atleast_1d(patterns)
        if argnums is None:
            argnums = np.arange(0, len(self._patterns))
        if len(self._patterns.shape) != 1:
            raise ValueError('patterns must be a 1d vector.')
        self._argnums = np.atleast_1d(argnums)
        self._argnum_sort = np.argsort(self._argnums)
        self.free = np.broadcast_to(free, self._patterns.shape)

        self._validate_args()
        
    def _validate_args(self):
        if len(self._argnums.shape) != 1:
            raise ValueError('argnums must be a 1d vector.')
        if len(self._argnums) != len(self._patterns):
            raise ValueError('argnums must be the same length as patterns.')
        if len(self.free.shape) != 1:
            raise ValueError('free must be a single boolean or a 1d vector of booleans.')
        if len(self.free) != len(self._patterns):
            raise ValueError('free must broadcast to the same shape as patterns.')
        
    def __str__(self):
        return('Function: {}\nargnums: {}\nfree: {}\npatterns: {}'.format(
            self._fun, self._argnums, self.free, self._patterns))
    
    def __call__(self, *args, **kwargs):
        # Loop through the arguments from beginning to end, replacing parameters
        # with their flattened values.
        new_args = ()
        last_argnum = 0
        for i in self._argnum_sort:
            argnum = self._argnums[i]
            folded_val = self._patterns[i].fold(args[argnum], free=self.free[i])
            new_args += args[last_argnum:argnum] + (folded_val, )
            last_argnum = argnum + 1
            
        return self._fun(*new_args, **kwargs)


def test_function(x, a, y, b):
    return x ** 2 + y ** 2 + np.mean(a ** 2) + np.mean(b ** 2)


x = 2
y = 3
a = param_dict['a']
b = param_dict['b']
a_flat = dict_pattern['a'].flatten(a, True)
b_flat = dict_pattern['b'].flatten(b, True)


patterns = [dict_pattern['a'], dict_pattern['b']]
flat_test_function = FlattenedFunction(
    test_function, patterns, free=[True, True], argnums=[1, 3])

assert(np.linalg.norm(
    test_function(x, a, y, b) - flat_test_function(x, a_flat, y, b_flat) < 1e-9))

print(flat_test_function)




def test_function(param_dict):
    a = param_dict['a']
    b = param_dict['b']
    return np.mean(a ** 2) + np.mean(b ** 2)

flat_test_function = FlattenedFunction(
    test_function, dict_pattern, free=True)
param_dict_flat = dict_pattern.flatten(param_dict, free=True)
assert(np.linalg.norm(
    test_function(param_dict) - flat_test_function(param_dict_flat) < 1e-9))



Function: <function test_function at 0x7f362a34cae8>
argnums: [1 3]
free: [ True  True]
patterns: [<paragami.numeric_array_patterns.NumericArrayPattern object at 0x7f362a368278>


In [8]:
import datetime
np.array([datetime.datetime.now()])

array([array(datetime.datetime(2018, 10, 25, 15, 40, 26, 10484), dtype=object)],
      dtype=object)

In [9]:
a = np.random.random((2, 2))
print('a: ', a)

shape = (3, 4)
foo = np.empty(shape, dtype='object')
for i in range(3):
    for j in range(4):
        foo[i, j] = a
print(foo.shape)
print(foo[0, 1])

a:  [[0.37173823 0.37938577]
 [0.75542049 0.90285716]]
(3, 4)
[[0.37173823 0.37938577]
 [0.75542049 0.90285716]]


In [10]:
a = np.random.random((2, 2))
print(a)
shape = (3, 4)
#foo = np.reshape(np.array([a for i in range(np.prod(shape))]), shape + a.shape)
foo = np.array([a for i in range(np.prod(shape))])
print(foo.shape)
print(foo.dtype)
print('With shape broadcasting:\n', foo[0, 0])

print('Of a dict:')
a = dict(a=5)
shape = (3, 4)
foo = np.reshape(np.array([a for i in range(np.prod(shape))], dtype='object'), shape)
print(foo.shape)
print(foo[0, 0])


print('\nOf an object array:')
a = np.array(np.random.random((2, 2)), dtype='object')
print(a)
shape = (3, 4)
foo = np.reshape(np.array([a for i in range(np.prod(shape))]), shape + a.shape)
print(foo.shape)
print(foo.dtype)


[[0.95159685 0.56446536]
 [0.83483776 0.77302476]]
(12, 2, 2)
float64
With shape broadcasting:
 [0.95159685 0.56446536]
Of a dict:
(3, 4)
{'a': 5}

Of an object array:
[[0.2466555802189665 0.443340827643413]
 [0.7321678304150749 0.0741130893572356]]
(3, 4, 2, 2)
object


In [11]:
def is_np(a):
    return(type(a) is np.ndarray)

print(is_np(np.random.random((2, 2))))

a = np.array(np.random.random((2, 2)), dtype='object')
print(is_np(a))

print(is_np(dict(a=4)))


True
True
False


In [13]:
# Array
from paragami.base_patterns import PatternArray

pattern_array = PatternArray(
    (2, 3), paragami.NumericArrayPattern(shape=(2, ), lb=-1, ub=10.0))

val = pattern_array.random()
print(val.shape)
print(val)

pattern_array.flatten(val, True)
pattern_array.flatten(val, False)



(2, 3, 2)
[[[4.9039154  4.99681502]
  [6.23815611 5.94067709]
  [6.77417041 6.85706096]]

 [[6.7673582  6.15262281]
  [7.02905015 5.34491116]
  [6.94208903 5.76084896]]]


array([4.9039154 , 4.99681502, 6.23815611, 5.94067709, 6.77417041,
       6.85706096, 6.7673582 , 6.15262281, 7.02905015, 5.34491116,
       6.94208903, 5.76084896])